In [2]:
import numpy as np
import scipy.sparse as sps
import h5py
from termcolor import colored
import os

def decode_str(hdf5_str):
    return hdf5_str.tobytes().decode('utf-16')

filename = 'data/Head-and-Neck_02.mat'
f = h5py.File(filename)
name = decode_str(f['patient']['Identifier'][:])
print(colored('Patient: ' + name, 'blue'))

folder = filename.split('.')[0]
if not os.path.exists(folder):
    os.makedirs(folder)

Patient: Head-and-Neck 02


In [3]:
class Region:
    def __init__(self, name):
        self.name = name
        self.D = None # Dose deposition matrix (called A in TROTS)

    def compute_dose(self, x):
        self.dose = self.D.dot(x)
        self.min = self.dose.min()
        self.mean = self.dose.mean()
        self.max = self.dose.max()

def load_rois():
    print('Loading dose deposition matrices...')
    
    rois = {}
    for ref in f['patient']['StructureNames'][:]:
        name = decode_str(f[ref[0]][:])
        rois[name] = Region(name)

    # Get the dose matrix for each ROI
    n_mats = f['data']['matrix']['A'].shape[0]
    for i in range(n_mats):
        name = decode_str(f[f['data']['matrix']['Name'][i,0]][:])

        if name in rois:
            roi = rois[name]
            A = f[f['data']['matrix']['A'][i,0]]
            
            if isinstance(A, h5py.Group): # It's a sparse matrix
                data = np.array(A['data']).ravel()
                ir = np.array(A['ir']).ravel()
                jc = np.array(A['jc']).ravel()
                n_voxels = A.attrs.get('MATLAB_sparse')
                n_beamlets = jc.size - 1
                shape = (n_voxels, n_beamlets)
                
                roi.D = sps.csc_matrix((data, ir, jc), shape=shape)
                
                print(colored(f'  {name} -> {shape} (sparse)', 'green'))
            elif isinstance(A, h5py.Dataset): # It's a dense matrix
                roi.D = A[:].T
                shape = roi.D.shape
                print(colored(f'  {name} -> {shape} (dense)', 'green'))
            else:
                print(colored(f'  {name} -> Not processed!', 'magenta'))
        else:
            print(colored(f'  {name} -> Ignored', 'yellow'))

    # Check that all regions have dose matrices
    for name, roi in rois.items():
        if roi.D is None:
            print(colored(f'  {name} -> No dose matrix!', 'red'))
    
    return rois

rois = load_rois()

Loading dose deposition matrices...
  PTV 0-46 Gy -> (5167, 6331) (dense)
  Spinal Cord -> (3181, 6331) (sparse)
  Brainstem -> (3126, 6331) (sparse)
  PTV Shell 15 mm -> (4724, 6331) (sparse)
  PTV Shell 30 mm -> (4729, 6331) (sparse)
  PTV Shell 40 mm -> (4787, 6331) (sparse)
  PTV Shell 5 mm -> (4839, 6331) (sparse)
  Patient -> (10600, 6331) (sparse)
  PTV Shell 0 mm -> (4908, 6331) (sparse)
  Smoothing Linear -> Ignored
  Smoothing Quadratic -> Ignored
  Parotid (R) -> (3315, 6331) (sparse)
  Parotid (R) (mean) -> Ignored
  Parotid (L) -> (3165, 6331) (sparse)
  Parotid (L) (mean) -> Ignored
  SMG (R) -> (1487, 6331) (dense)
  SMG (R) (mean) -> Ignored
  SMG (L) -> (1675, 6331) (sparse)
  SMG (L) (mean) -> Ignored
  Oral Cavity -> (5325, 6331) (sparse)
  Oral Cavity (mean) -> Ignored
  External Ring 20 mm -> (5409, 6331) (sparse)
  Larynx -> (5075, 6331) (dense)
  Larynx (mean) -> Ignored
  MCS -> (1621, 6331) (dense)
  MCS (mean) -> Ignored
  MCM -> (990, 6331) (dense)
  MCM (mea

In [5]:
import os

def n_voxels(rois):
    n_voxels = 0
    for name, roi in rois.items():
        n_voxels += roi.D.shape[0]
    return n_voxels

def gen_m(folder):
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    with open(os.path.join(folder, 'm_' + name + '.txt'), 'w') as file:
        file.write(name + '\n')
        # We don't care about beam geometry for now.
        file.write('1 // Number of beams \n')
        file.write('1 {} \n'.format(int(f['data']['misc']['size'][0][0])))
        file.write('{} // Number of voxels \n'.format(n_voxels(rois)))
        # d_ files will hold float values without need of scaling.
        file.write('1.0 // DoseGridScaling \n'.format(n_voxels(rois)))
        file.write('{} // Number of ROIs \n'.format(len(rois)))
        for i, (name, roi) in enumerate(rois.items()):
            file.write('{} {}\n'.format(2**i, name))
            

def gen_v(folder):
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    with open(os.path.join(folder, 'v_' + name + '.txt'), 'w') as file:
        for i, (name, roi) in enumerate(rois.items()):
            print(name)
            voxel_region = 2**i
            for j in range(roi.D.shape[0]):
                file.write('{}\n'.format(voxel_region))
                
gen_m(folder)
gen_v(folder)

Patient
Spinal Cord
Parotid (R)
Parotid (L)
SMG (R)
SMG (L)
MCS
MCM
MCI
MCP
Oesophagus
Brainstem
Oral Cavity
Larynx
PTV 0-46 Gy
PTV Shell 15 mm
PTV Shell 30 mm
PTV Shell 40 mm
PTV Shell 5 mm
PTV Shell 0 mm
External Ring 20 mm


In [6]:
def gen_d(folder):
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    
    total_nnz = 0
    for n, roi in rois.items():
        coo = sps.coo_matrix(roi.D)
        total_nnz += coo.nnz
    
    with open(os.path.join(folder, 'd_' + name + '.txt'), 'w') as file:
        file.write('{} \n'.format(total_nnz))
        row_offset = 0
        for i, (n, roi) in enumerate(rois.items()):
            coo = sps.coo_matrix(roi.D)
            print(f'[{i+1:02d}/{len(rois):02d}] {n}: {coo.nnz} nonzeros...')
            for j in range(coo.nnz):
                file.write('{} {} {:.8f}\n'.format(coo.row[j] + row_offset, coo.col[j], coo.data[j]))
            row_offset += coo.shape[0]

gen_d(folder)

[01/21] Patient: 9427833 nonzeros...
[02/21] Spinal Cord: 3796857 nonzeros...
[03/21] Parotid (R): 3124502 nonzeros...
[04/21] Parotid (L): 2700168 nonzeros...
[05/21] SMG (R): 2453902 nonzeros...
[06/21] SMG (L): 2466322 nonzeros...
[07/21] MCS: 2628859 nonzeros...
[08/21] MCM: 2030729 nonzeros...
[09/21] MCI: 2598988 nonzeros...
[10/21] MCP: 2877210 nonzeros...
[11/21] Oesophagus: 3074332 nonzeros...
[12/21] Brainstem: 356026 nonzeros...
[13/21] Oral Cavity: 4542374 nonzeros...
[14/21] Larynx: 9824240 nonzeros...
[15/21] PTV 0-46 Gy: 8790518 nonzeros...
[16/21] PTV Shell 15 mm: 5590849 nonzeros...
[17/21] PTV Shell 30 mm: 3610265 nonzeros...
[18/21] PTV Shell 40 mm: 2336881 nonzeros...
[19/21] PTV Shell 5 mm: 7093866 nonzeros...
[20/21] PTV Shell 0 mm: 7750010 nonzeros...
[21/21] External Ring 20 mm: 912145 nonzeros...


In [8]:
def gen_x():
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    fluence = f['solutionX'][:].ravel()
    
    with open(os.path.join(folder, 'x_' + name + '.txt'), 'w') as file:
        for x in fluence:
            file.write('{}\n'.format(x))

gen_x()